In [1]:
import numpy as np
import pandas as pd
import glob
import ntpath

In [2]:
names = ['HUC_02','GAGE_ID','GAGE_NAME','LATITUDE','LONGITUDE','ELEVATION']
gauges = pd.read_table('/Volumes/data/basin_dataset_public/basin_metadata/gauge_information.txt',usecols=[0,1,2,3,4,5],names=names,skiprows=1)

In [3]:
names = ['HUC_02','GAGE_ID','AREA','Elevation','SLOPE','PERC_FOREST']
basins = pd.read_table('/Volumes/data/basin_dataset_public/basin_metadata/basin_physical_characteristics.txt',delim_whitespace=True,skiprows=0,names=names,header=False)

In [4]:
data = pd.merge(gauges,basins,on='GAGE_ID')

In [5]:
data['DAYMET_P'] = np.NaN
data['MAURER_P'] = np.NaN
data['NLDAS_P'] = np.NaN

In [6]:
daymet = glob.glob('/Volumes/data/basin_dataset_public/basin_mean_forcing/daymet/*/*.txt')

In [7]:
names = ['year','month','day','hour','dayl','P','srad','SWE','tmax','tmin','vp']

for dspth in daymet:
    ds = pd.read_table(dspth,skiprows=3,names=names, delim_whitespace=True,header=False)

    path,basename = ntpath.split(dspth)
    bsn = int(basename.split('_')[0])

    data.loc[data.GAGE_ID==bsn,'DAYMET_P'] = ds.P.sum() # [mm]

In [8]:
maurer = glob.glob('/Volumes/data/basin_dataset_public/basin_mean_forcing/maurer/*/*.txt')

In [9]:
names = ['year','month','day','hour','dayl','P','srad','SWE','tmax','tmin','vp']

for dspth in maurer:
    ds = pd.read_table(dspth,skiprows=3,names=names, delim_whitespace=True,header=False)

    path,basename = ntpath.split(dspth)
    bsn = int(basename.split('_')[0])

    data.loc[data.GAGE_ID==bsn,'MAURER_P'] = ds.P.sum() # [mm]

In [10]:
nldas = glob.glob('/Volumes/data/basin_dataset_public/basin_mean_forcing/nldas/*/*.txt')

In [11]:
names = ['year','month','day','hour','dayl','P','srad','SWE','tmax','tmin','vp']

for dspth in nldas:
    ds = pd.read_table(dspth,skiprows=3,names=names, delim_whitespace=True,header=False)

    path,basename = ntpath.split(dspth)
    bsn = int(basename.split('_')[0])

    data.loc[data.GAGE_ID==bsn,'NLDAS_P'] = ds.P.sum() # [mm]

## Bring in the Discharge data and Compute the runoff ratios

In [16]:
strm = pd.read_pickle('../data/processed_observed_streamflow_sums.pcl')

In [20]:
strm.columns = ['Q','GAGE_ID']

In [21]:
data = pd.merge(data,strm,on='GAGE_ID')

In [22]:
data.head()

,HUC_02_x,GAGE_ID,GAGE_NAME,LATITUDE,LONGITUDE,ELEVATION,HUC_02_y,AREA,Elevation,SLOPE,PERC_FOREST,DAYMET_P,MAURER_P,NLDAS_P,Q
0,1,1013500,"Fish River near Fort Kent, M...",47.23739,-68.58264,2252.70,1,2303.95,250.31,21.64152,0.9063,36456.73,28607.88,29348.66,18425.027697
1,1,1022500,"Narraguagus River at Cherryfield, M...",44.60797,-67.93524,573.60,1,620.38,92.68,17.79072,0.9232,41918.23,35027.79,33562.56,22496.532338
2,1,1030500,"Mattawamkeag River near Mattawamkeag, M...",45.50097,-68.30596,3676.17,1,3676.09,143.80,12.79195,0.8782,37845.36,31471.68,32952.08,20261.683804
3,1,1031500,"Piscataquis River near Dover-Foxcroft, M...",45.17501,-69.31470,769.05,1,766.53,247.80,29.56035,0.9548,40376.67,33670.00,34245.20,22637.790215
4,1,1047000,"Carrabassett River near North Anson, M...",44.86920,-69.95510,909.10,1,904.94,310.38,49.92122,0.9906,38581.42,33415.72,36097.74,24011.176958


In [23]:
data['DAYMET_RR'] = data.Q/data.DAYMET_P
data['MAURER_RR'] = data.Q/data.MAURER_P
data['NLDAS_RR'] = data.Q/data.NLDAS_P

In [24]:
data.to_pickle('../data/newman_RRs.pcl')